In [1]:
%%python

UsageError: %%python is a cell magic, but the cell body is empty.


In [2]:
sc.stop()

In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
sc = SparkContext.getOrCreate()
#
spark = SparkSession(sc)

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
#sc = SparkContext.getOrCreate()
#spark = SparkSession(sc)

In [5]:
hats2 = spark.read.csv("hats_data.csv", header="true", inferSchema="true")

In [6]:
pants = spark.read.csv("pants_data.csv", header="true", inferSchema="true")

In [7]:
#pants = pants.select("size","cost","color").filter(col("color") == "blue")

In [8]:
pants.show(5)

+-----+----+----+
|color|size|cost|
+-----+----+----+
|    1|   1| 106|
|    2|   1|  73|
|    3|   1|  57|
|    3|   2|  39|
|    2|   1| 104|
+-----+----+----+
only showing top 5 rows



STARTING ML

In [9]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler


with assembler

In [10]:
vectorAssembler = VectorAssembler(inputCols = ['color', 'size'], outputCol = 'features')

In [11]:
pants = vectorAssembler.transform(pants)
pants = pants.select(['features', 'cost'])

In [12]:
data = pants

In [13]:
data.show(5)

+---------+----+
| features|cost|
+---------+----+
|[1.0,1.0]| 106|
|[2.0,1.0]|  73|
|[3.0,1.0]|  57|
|[3.0,2.0]|  39|
|[2.0,1.0]| 104|
+---------+----+
only showing top 5 rows



In [14]:
train_df, test_df  = data.randomSplit([0.7, 0.3])

In [15]:
test_df.show(5)

+---------+----+
| features|cost|
+---------+----+
|[1.0,1.0]| 106|
|[1.0,1.0]| 136|
|[1.0,1.0]| 166|
|[1.0,1.0]| 171|
|[1.0,1.0]| 179|
+---------+----+
only showing top 5 rows



In [17]:
# Import LinearRegression class
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
lr = LinearRegression(featuresCol = 'features', labelCol='cost', maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit 2 models, using different regularization parameters
modelA = lr.fit(train_df)
#modelB = lr.fit(tdata, {lr.regParam:100.0})

In [18]:
# Make predictions
predictionsA = modelA.transform(test_df)
display(predictionsA)

DataFrame[features: vector, cost: int, prediction: double]

In [19]:
predictionsA.show(100)

+---------+----+------------------+
| features|cost|        prediction|
+---------+----+------------------+
|[1.0,1.0]| 106| 79.49577944200186|
|[1.0,1.0]| 136| 79.49577944200186|
|[1.0,1.0]| 166| 79.49577944200186|
|[1.0,1.0]| 171| 79.49577944200186|
|[1.0,1.0]| 179| 79.49577944200186|
|[1.0,2.0]|  30| 154.1521462284602|
|[1.0,2.0]| 182| 154.1521462284602|
|[1.0,2.0]| 184| 154.1521462284602|
|[2.0,1.0]|  34|102.80638875345979|
|[2.0,2.0]|  47| 177.4627555399181|
|[2.0,2.0]| 170| 177.4627555399181|
|[3.0,1.0]|  57|126.11699806491772|
|[3.0,1.0]| 121|126.11699806491772|
|[3.0,1.0]| 134|126.11699806491772|
|[3.0,1.0]| 181|126.11699806491772|
|[3.0,2.0]|  39|200.77336485137602|
|[3.0,3.0]| 288| 275.4297316378344|
|[3.0,4.0]| 174| 350.0860984242927|
|[3.0,4.0]| 373| 350.0860984242927|
|[3.0,5.0]| 546|  424.742465210751|
|[4.0,1.0]|  28|149.42760737637562|
|[4.0,1.0]|  28|149.42760737637562|
|[4.0,1.0]|  39|149.42760737637562|
|[4.0,1.0]|  39|149.42760737637562|
|[4.0,1.0]|  88|149.42760737

In [20]:
trainingSummary = modelA.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 147.901822
r2: 0.426089
